# Concat xls files to single csv file

In [1]:
DATA_PATH = "../data/ข้อมูลน้ำ 58-63"
OUTPUT_PATH = "../output/raw-csv"

In [ ]:
from pathlib import Path
from pandas import DataFrame, read_excel, to_datetime, concat
from logging import basicConfig, INFO

basicConfig(level=INFO)

from mp import mp_print, mp_exec

INFO:mp:Set start method to fork


In [3]:
DATA_PATH = Path(DATA_PATH).resolve()
OUTPUT_PATH = Path(OUTPUT_PATH).resolve()
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

In [4]:
def iter_files():
    for path in filter(lambda x: x.is_dir(), DATA_PATH.iterdir()):
        for station in filter(lambda x: x.is_dir(), path.iterdir()):
            for file in filter(lambda file: file.is_file() and file.suffix == ".xls", station.iterdir()):
                yield (f"{path.name} {station.name}", file)

In [5]:
MONTH = {x: y for x, y in zip("ม.ค. ก.พ. มี.ค. เม.ย. พ.ค. มิ.ย. ก.ค. ส.ค. ก.ย. ต.ค. พ.ย. ธ.ค.".split(), range(1, 13))}

def to_datetime_cb(dt: str):
    day, month_year, _time = dt.split(" ")
    return to_datetime(f"{int(month_year[-4:]) - 543}-{MONTH[month_year[:-4]]}-{day} {_time[0]}:{_time[1]}")

In [ ]:
def task(args: tuple[str, Path]):
    station, file = args
    mp_print(f"Processing : {station} - {file.name}")

    df = read_excel(file)
    df["Datetime"] = df["วันที่-เวลา"].apply(to_datetime_cb)
    df.drop(columns=["วันที่-เวลา", "หมายเลข"], inplace=True)
    df.replace('-', '', inplace=True)
    df = df[["Datetime"] + [col for col in df.columns if col != "Datetime"]]
    df.sort_values(by="Datetime", inplace=True)

    return (station, df)

In [7]:
print(f"Collecting data...")

data: dict[str, DataFrame] = {}

for station, df in mp_exec(task, iter_files(), unorder=True):
    if station not in data:
        data[station] = df
    else:
        data[station] = concat([data[station], df])

print("Collected data, number of stations : ", len(data))

Processing : ป่าสัก3สถานี นครหลวง 2558-2563 - นครหลวง61-กค-ธค.xls
Processing : ป่าสัก3สถานี นครหลวง 2558-2563 - นครหลวง60-มค-มิย.xls
Processing : ป่าสัก3สถานี นครหลวง 2558-2563 - นครหลวง59-มค-มิย.xls
Processing : ป่าสัก3สถานี นครหลวง 2558-2563 - นครหลวง62-มค-มิย.xls
Processing : ป่าสัก3สถานี นครหลวง 2558-2563 - นครหลวง60-กค-ธค.xls
Processing : ป่าสัก3สถานี นครหลวง 2558-2563 - นครหลวง59-กค-ธค.xls
Processing : ป่าสัก3สถานี นครหลวง 2558-2563 - นครหลวง63-มค-มิย.xls
Processing : ป่าสัก3สถานี นครหลวง 2558-2563 - นครหลวง61-มค-มิย.xls
Processing : ป่าสัก3สถานี นครหลวง 2558-2563 - นครหลวง62-กค-ธค.xls
Processing : ป่าสัก3สถานี แก่งคอย 2558-2563 - แก่งคอย60-มค-มิย.xls
Processing : ป่าสัก3สถานี นครหลวง 2558-2563 - นครหลวง63-กค-ธค.xls
Processing : ป่าสัก3สถานี นครหลวง 2558-2563 - นครหลวง58-กพ-มิย.xls
Processing : ป่าสัก3สถานี แก่งคอย 2558-2563 - แก่งคอย63-กค-ธค.xls
Processing : ป่าสัก3สถานี แก่งคอย 2558-2563 - แก่งคอย59-กค-ธค.xls
Processing : ป่าสัก3สถานี แก่งคอย 2558-2563 - แก่งคอย58-กค-ธค.xls
Pro

In [ ]:
for station, df in data.items():
    print(f"Station : {station}, data : {df.shape}")

    data[station].sort_values(by="Datetime", inplace=True)
    df.to_csv(OUTPUT_PATH / f"{station}.csv", index=False)

Station : ป่าสัก3สถานี นครหลวง 2558-2563, data : (85184, 9)
Station : ป่าสัก3สถานี แก่งคอย 2558-2563, data : (92688, 6)
Station : ป่าสัก3สถานี เสาไห้ 2558-2563, data : (70819, 9)
Station : ยม สุโขทัย 2558-2563, data : (28469, 5)
Station : ยม สามง่าม 2558-2563, data : (77859, 5)
Station : ยม โพทะเล 2558-2563, data : (54607, 6)
Station : วัง เกาะคา 2558-2563, data : (95369, 5)
Station : แม่น้ำท่าจีน กระทุ่มแบน 2558-2563, data : (71976, 8)
Station : แม่น้ำท่าจีน บางเลน 2558-2563, data : (98746, 9)
Station : แม่น้ำท่าจีน สองพี่น้อง 2558-2563, data : (74837, 6)
Station : แม่น้ำท่าจีน นครชัยศรี 2558-2563, data : (81149, 7)
Station : แม่น้ำท่าจีน หันคา 2558-2563, data : (72458, 5)
Station : แม่น้ำท่าจีน สุพรรณบุรี 2558-2563, data : (77994, 9)
Station : แม่น้ำท่าจีน สามชุก 2558-2563, data : (30737, 9)
Station : ปิง เชียงใหม่ 2558-2563, data : (83708, 5)
Station : ปิง กำแพงเพชร 2558-2563, data : (82788, 5)
Station : น่าน อุตรดิตถ์ 2558-2563, data : (96165, 5)
Station : น่าน น่าน 2558-2563, data